<a href="https://colab.research.google.com/github/TalesMiguel/RNA/blob/main/trabalho3_VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Redes Neurais Artificiais - Trabalho 3: Autoencoders Variacionais (VAE)**

## Feito por:
Camilo Maia Pires - 140473
Tales Miguel Machado Pereira - 140247

## **Introdução**


### Autoencoders Variacionais (VAE)

Autoencoders Variacionais (VAEs) são modelos generativos que combinam redes neurais com inferência variacional para aprender representações latentes de dados. Eles são compostos por duas partes principais: o **encoder**, que mapeia os dados de entrada para um espaço latente, e o **decoder**, que reconstrói os dados a partir desse resultado. A principal diferença entre VAEs e autoencoders tradicionais é que VAEs aprendem uma distribuição de probabilidade no espaço latente estruturado e contínuo.

<br>

### Datasets Utilizados

Para este trabalho, foram selecionados dois datasets rotulados:

1. **Digits Dataset**: Um conjunto de dados que contém imagens de dígitos manuscritos (0 a 9). Cada imagem é representada por uma matriz 8x8 de pixels, resultando em 64 features.

2. **Iris Dataset**: Um conjunto de dados clássico que contém 150 amostras de flores Iris, com 4 features cada (comprimento e largura das sépalas e pétalas) e 3 classes (Setosa, Versicolour e Virginica).

<br>

### Objetivo

O objetivo deste trabalho é treinar modelos VAEs nesses datasets, explorar o espaço latente gerado e analisar a formação de clusters e a separação dos rótulos nesse espaço. Além disso, será avaliado quanto da variância dos dados é capturada pela projeção do espaço latente em 2D usando PCA.

Adicionalmente, tentaremos enviesar a formação do espaço latente com os exemplos previamente rotulados.

<br>


---



## Implementação

### Importação de Bibliotecas



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers, models, losses
from tensorflow.keras import backend as K
import tensorflow as tf

### Carregamento dos Datasets

In [ ]:
# Digits Dataset
digits = datasets.load_digits()
X_digits = digits.data      # Features
y_digits = digits.target    # Rótulos

# Iris Dataset
iris = datasets.load_iris()
X_iris = iris.data          # Features
y_iris = iris.target        # Rótulos

### Pré-processamento dos Dados

Antes de treinar o VAE, precisamos normalizar os dados para garantir que todas suas features estejam na mesma escala.

In [ ]:
X_digits_scaled = StandardScaler().fit_transform(X_digits)
X_iris_scaled = StandardScaler().fit_transform(X_iris)

### Construção do Modelo VAE

A seguir, definimos a arquitetura do VAE. O encoder mapeia os dados de entrada para o espaço latente, enquanto o decoder tenta reconstruir os dados a partir desse espaço.

In [ ]:
# Parâmetros do VAE
input_dim_digits = X_digits_scaled.shape[1]
input_dim_iris = X_iris_scaled.shape[1]
latent_dim = 2  # Dimensão do espaço latente

# Função de perda do VAE
def vae_loss(x, x_decoded_mean):
    reconstruction_loss = losses.mse(x, x_decoded_mean)
    kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    return K.mean(reconstruction_loss + kl_loss)

# Encoder
inputs = layers.Input(shape=(input_dim_digits,))
x = layers.Dense(128, activation='relu')(inputs)
z_mean = layers.Dense(latent_dim)(x)
z_log_var = layers.Dense(latent_dim)(x)

# Amostragem no espaço latente
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0., stddev=1.)
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_var])

# Decoder
decoder_input = layers.Input(shape=(latent_dim,))
x = layers.Dense(128, activation='relu')(decoder_input)
outputs = layers.Dense(input_dim_digits, activation='sigmoid')(x)

# Modelos
encoder = models.Model(inputs, [z_mean, z_log_var, z], name='encoder')
decoder = models.Model(decoder_input, outputs, name='decoder')
vae_outputs = decoder(encoder(inputs)[2])
vae = models.Model(inputs, vae_outputs, name='vae')

# Compilação do modelo
vae.compile(optimizer='adam', loss=vae_loss)


### Treinamento do VAE

Treinamos o VAE no Digits Dataset e no Iris Dataset.


In [ ]:
# Treinamento no Digits Dataset
vae.fit(X_digits_scaled, X_digits_scaled, epochs=50, batch_size=32, validation_split=0.2)

# Treinamento no Iris Dataset
vae.fit(X_iris_scaled, X_iris_scaled, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50


ValueError: Tried to convert 'x' to a tensor and failed. Error: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```


### Exploração do Espaço Latente

Após o treinamento, exploramos o espaço latente gerado pelo VAE. Para isso, projetamos o espaço latente em 2D usando PCA e visualizamos a formação de clusters.


In [ ]:
# Projeção do espaço latente em 2D usando PCA
def plot_latent_space(encoder, X, y, title):
    z_mean, _, _ = encoder.predict(X)
    pca = PCA(n_components=2)
    z_pca = pca.fit_transform(z_mean)

    plt.figure(figsize=(8, 6))
    plt.scatter(z_pca[:, 0], z_pca[:, 1], c=y, cmap='viridis')
    plt.colorbar()
    plt.title(title)
    plt.xlabel('Componente Principal 1')
    plt.ylabel('Componente Principal 2')
    plt.show()

# Plot para o Digits Dataset
plot_latent_space(encoder, X_digits_scaled, y_digits, 'Espaço Latente - Digits Dataset')

# Plot para o Iris Dataset
plot_latent_space(encoder, X_iris_scaled, y_iris, 'Espaço Latente - Iris Dataset')

### Análise dos Resultados

1. **Formação de Clusters**: No espaço latente, observamos a formação de clusters para ambos os datasets. No caso do Digits Dataset, os clusters correspondem aos diferentes dígitos, enquanto no Iris Dataset, os clusters correspondem às diferentes espécies de flores.

2. **Separação dos Rótulos**: A separação dos rótulos no espaço latente é evidente, especialmente no Iris Dataset, onde as três espécies de flores são bem separadas. No Digits Dataset, alguns dígitos se sobrepõem, o que pode ser devido à similaridade entre eles (por exemplo, os dígitos 1 e 7).

3. **Variância Capturada**: A projeção PCA do espaço latente captura uma quantidade significativa da variância dos dados, como evidenciado pela clara separação dos clusters.

### Conclusão

Neste trabalho, treinamos modelos VAEs em dois datasets diferentes e exploramos o espaço latente gerado. Observamos a formação de clusters e a separação dos rótulos no espaço latente, além de avaliar a variância capturada pela projeção PCA. Os resultados demonstram a eficácia dos VAEs em aprender representações latentes significativas dos dados.

---

### Adicional (Opcional): Enviesar a Formação do Espaço Latente

Para enviesar a formação do espaço latente com os exemplos rotulados, podemos incorporar os rótulos no processo de treinamento do VAE. Isso pode ser feito modificando a função de perda para incluir uma componente que penaliza a distância entre as amostras do mesmo rótulo no espaço latente.

# Função de perda modificada para enviesar o espaço latente

In [ ]:
def vae_loss_with_labels(x, x_decoded_mean):
    reconstruction_loss = losses.mse(x, x_decoded_mean)
    kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    label_loss = K.mean(K.square(z_mean[y == y] - z_mean[y == y]))  # Penaliza distância entre amostras do mesmo rótulo
    return K.mean(reconstruction_loss + kl_loss + label_loss)
